In [1]:
import pickle

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import utils
import calibration_utils

import importlib
importlib.reload(utils)
importlib.reload(calibration_utils)

<module 'calibration_utils' from '/home/tom/Documents/UNT/csce6260/projects/pcdet_notebooks/calibration_utils.py'>

In [3]:
CLASS_COLORS = {
    1: (1, 0, 0),    # Car     (red)
    2: (0, 1, 0),    # Pedestrian (green)
    3: (0, 0, 1)     # Cyclist (blue)
}

def get_point_colors(z):
    z_norm = (z - z.min()) / (z.max() - z.min() + 1e-6)
    colors = plt.cm.jet(z_norm)[:, :3]
    return colors

def create_point_cloud(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    colors = get_point_colors(points[:, 2])
    pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd

def get_min_and_max_values(points):
    min_x, max_x = np.min(points[:, 0]), np.max(points[:, 0])
    min_y, max_y = np.min(points[:, 1]), np.max(points[:, 1])
    min_z, max_z = np.min(points[:, 2]), np.max(points[:, 2])
    return min_x, max_x, min_y, max_y, min_z, max_z

### Augmented Data

In [4]:
with open("data/sample_000000.pkl", "rb") as f:
    sample = pickle.load(f)

print("Sample keys:", sample.keys())

Sample keys: dict_keys(['frame_id', 'gt_boxes', 'points', 'flip_x', 'noise_rot', 'noise_scale', 'lidar_aug_matrix', 'use_lead_xyz', 'image_shape'])


In [5]:
points = sample['points']

print("Points shape:", points.shape)
print('Min/Max values for all dimensions')
min_x, max_x, min_y, max_y, min_z, max_z = get_min_and_max_values(points)
print(f"X: ({min_x:.2f}, {max_x:.2f})")
print(f"Y: ({min_y:.2f}, {max_y:.2f})")
print(f"Z: ({min_z:.2f}, {max_z:.2f})")

Points shape: (16384, 4)
Min/Max values for all dimensions
X: (5.25, 63.28)
Y: (-15.11, 29.17)
Z: (-2.43, 2.02)


In [6]:
def get_gt_oriented_bounding_boxes(gt_boxes):
    boxes = []
    for box in gt_boxes:
        x, y, z, dx, dy, dz, heading, class_idx = box
        class_idx = int(class_idx)
        obb = o3d.geometry.OrientedBoundingBox(
            center=[0, 0, 0],
            R=o3d.geometry.get_rotation_matrix_from_xyz([0, 0, heading]),
            extent=[dx, dy, dz]
        )
        obb.translate([x, y, z])
        obb.color = CLASS_COLORS.get(class_idx, (1, 1, 1))
        boxes.append(obb)
    return boxes


points = sample['points'][:, :3]
pcd = create_point_cloud(points)
gt_boxes = get_gt_oriented_bounding_boxes(sample['gt_boxes'])
o3d.visualization.draw_geometries([pcd, *gt_boxes])

### Original Data

In [7]:
bin_path = "/home/tom/kitti/training/velodyne/000000.bin"
orig_points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)

print("Points shape:", orig_points.shape)
print('Min/Max values for all dimensions')
min_x, max_x, min_y, max_y, min_z, max_z = get_min_and_max_values(orig_points)
print(f"X: ({min_x:.2f}, {max_x:.2f})")
print(f"Y: ({min_y:.2f}, {max_y:.2f})")
print(f"Z: ({min_z:.2f}, {max_z:.2f})")

Points shape: (115384, 4)
Min/Max values for all dimensions
X: (-71.04, 73.04)
Y: (-21.10, 53.80)
Z: (-5.16, 2.67)


In [8]:
def get_labels(label_file_path):
    labels = utils.parse_label_file(label_file_path)
    return labels

def get_Tr_cam_to_velo(calib_file_path):
    calib = utils.parse_calib_file(calib_file_path)
    Tr_velo_to_cam = calib['Tr_velo_to_cam']
    Tr_velo_to_cam = np.vstack((Tr_velo_to_cam, [0, 0, 0, 1]))
    Tr_cam_to_velo = calibration_utils.inverse_rigid_transform(Tr_velo_to_cam)
    return Tr_cam_to_velo

def get_gt_oriented_bounding_boxes(labels, Tr_cam_to_velo):
    gt_oriented_boxes = []
    for label in labels:
        corners_3d_velo = calibration_utils.compute_box_3d(label, Tr_cam_to_velo)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(corners_3d_velo)
        oriented_bbox = pcd.get_oriented_bounding_box()
        oriented_bbox.color = (0, 1, 0)
        gt_oriented_boxes.append(oriented_bbox)
    return gt_oriented_boxes


orig_points = orig_points[:, :3]
orig_pcd = create_point_cloud(orig_points)

label_file_path = "/home/tom/kitti/training/label_2/000000.txt"
labels = get_labels(label_file_path)

calib_file_path = "/home/tom/kitti/training/calib/000000.txt"
Tr_cam_to_velo = get_Tr_cam_to_velo(calib_file_path)

gt_oriented_boxes = get_gt_oriented_bounding_boxes(labels, Tr_cam_to_velo)
o3d.visualization.draw_geometries([orig_pcd, *gt_oriented_boxes])